# Boost Your Boost 

### Making Your ML WorkFlow Bigger and Faster

In this notebook, we will cover:
1. **Zarr** and its `Array` object for efficient, parallel data storage and retrieval.
2. Parallelizing computations with **Dask**.
3. Combining **Dask** and **XGBoost** to enable parallel regression on big (for simplicity, medium) data.
4. Leveraging **Dask** to distribute hyperparameter search.
5. Saving and loading an **XGBoost** model for deployment in production code.

Prerequisite: A working understanding of the machine learning workflow, NumPy, and Scikit-Learn. 

### Importing Packages and Starting Our Client

The imports are fairly straightforward. However, note that `dask_ml` and `dask` are different packages and need to be installed separately.

More interestingly, we instantiate a Dask `Client`. Our `Client` manages scheduling, and generally attempts to optimize the parallelization of our code. Here, we have implicitly opened our client on our computers local cluster (cpu). To use a different cluster, such as a cloud-based, hpc, or gpu cluster, be sure to check out the [docs](https://docs.dask.org/en/stable/deploying.html).

Invoking `client` in a notebook prints information about the client, and gives us a link to the Dask dashboard. While some graphics in the dashboard are straightforward, it may help to [read](https://docs.dask.org/en/latest/dashboard.html) about everything being shown.

In [1]:
import zarr
import dask.array as da
import dask.distributed
import dask_ml.model_selection as dcv
import sklearn.model_selection as sklcv # necessary due to bug
import dask_ml.metrics as dmetrics
import xgboost as xgb

from math import pi as π

client = dask.distributed.Client()
client

# Writing a py script? Use this instead.
# if __name__ == '__main__':
#     with dask.distributed.Client() as client:
#         your dask code

/Users/chadgueli/miniconda3/envs/dayone/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52174,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:52193,Total threads: 4
Dashboard: http://127.0.0.1:52194/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:52178,


## Data Creation and Storage Writing

We work with a parallelized data creation routine, and store the data in a `.zarr` file to practice both of these techniques. An added bonus is that it is more realistic to load our data from a file. 

### Storage with Zarr

To make the data persistent, we open our `data.zarr` file as a `DirectoryStore`. That is, we use the `DirectoryStore` to act as if we are collecting and storing data for later use.
Zarr offers many other storage formats that allow us to succinctly load data from our storage type of choice. For more info about the storage options, see the [Zarr docs](https://zarr.readthedocs.io/en/stable/api/storage.html).

A great thing about Zarr files is that we are able to create in-file subdirectories called groups. In this way, Zarr files are like an easily movable file system focused on memory-and-speed-efficient array storage. Here we group the data into train and test sets.

Finally, we call the `close` method on `zstore` so that Dask can do parallel reads and writes.

**Why Zarr?** Some may read the preceding and think it sounds quite similar to [HDF5](https://www.h5py.org). Zarr has a similar interface to HDF5, but allows parallel reads and writes on the same array, giving it an advantage over HDF5 in the 2020s.

**A Note On Zarr** Only array data storage is possible, but Zarr supports all NumPy dtypes. So feel free to create a `U8` file and name it `char.zarr`(d). 

In [2]:
zstore = zarr.DirectoryStore('data.zarr')
zroot = zarr.group(store=zstore)
train, test = zroot.create_groups('train', 'test')

zstore.close()

### Parallel Data Creation with Dask.Array

We are generating the data from the challenging [“Friedman #1”](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_friedman1.html#sklearn.datasets.make_friedman1) regression problem, and adding some irrelevant features and noise for fun. Composing the sine function on two variables makes this challenge particularly difficult for tree based models because of the large number of nodes needed to represent the curves. Nevertheless, we will be using XGBoost later in this chance to exhibit the strength of this estimator, even when its sub-estimators are suboptimal.

Notably, we set the `chunks` arg while initializing our arrays. Chunks are sub-arrays stored as `numpy.ndarray` objects that Dask moves around in memory, and compresses when not in use. Chunks allow Dask to parallelize computation on arrays, even if they are too big to fit in memory. Dask will choose a default chunk size, but it may not be optimal in all use cases. For example, Dask recommends chunks of size at least 100 Mb, but our chunks are only 15.26 Mb because we found this size to be less-problematic with XGBoost.

**IMPORTANT** `chunks` is short for chunk size. We are providing a size, not the number of chunks. Since `None` is the second element of the tuple passed to `chunks`, there is no chunking across rows. Thus, `X` is 1,000,000 by 20 and has chunks of size 100,000 by 20. For more info on optimal chunking, see this [Dask tutorial](https://docs.dask.org/en/latest/array-chunks.html).

That's it, when this code is run, Dask will parallelize this computation for us, but it will not compute it yet. This code is actually telling dask to generate a task graph, which is Dask's particular name for a DAG; like how an `ndarray` is a tensor. Most of the Dask api is lazy, so it only runs computations when necessary. While `X` will be sampled and `y` will be executed in the next block, if we want to compute these values now and bring them into memory, we can call `.compute`.

A directed acyclic graph (DAG) is a representation of the program, where nodes are objects, and edges are actions that produce new objects. Also there are no cycles, so no computations may occur in-place. Dask uses the DAG model to schedule jobs and plan communication between workers. Calling `.visualize`, we can generate a picture of our DAG when the [graphviz](https://anaconda.org/anaconda/graphviz) package is installed. Since the graph for our data creation is massive, only the portion pertaining to the first chunk is included. Since the chunks don't interact, each chunk has the same DAG.

**Why Dask?** Some might say that similar functionality is available in [PySpark](https://spark.apache.org/docs/latest/api/python/index.html). Ultimately, PySpark is a module that converts code to another language (Scala) where it is compiled by Spark to a program that attempts to compile the parallel execution before finally running the code. Dask on the other hand uses python to create a graph that is then run using the python interpreter. Neither option is great as both provide partial programming languages hidden in another language, or in PySpark's case two other languages. Optimally, we would use Julia which walks like Python, runs like C, and provides meta-programming that allows the DAG to be created on any type of object from anywhere.

In [3]:
N_OBS = 10_000_000
N_FTS = 20
N_TRN = 4*N_OBS//5
dim1 = 100_000

X = da.random.uniform(size=(N_OBS, N_FTS), chunks=(dim1, None))
y = 2*da.sin(π*X[:, 0] * X[:, 1]) + 4*(X[:, 2] - 0.5)**2 + 2*X[:, 3] + X[:, 4]
y = 5*y + 0.1*da.random.normal(size=N_OBS, chunks=dim1)

#y.visualize()

![Data Creation DAG on 500k Observations](chunkdag.png)

With the file structure set up, we write to our four Zarr arrays with Dask's `to_zarr` function. Again, we previously closed `data.zarr` because Dask won't write to arrays in memory.
Zarr also chunks its arrays; using chunks as units of compression allows Zarr to keep the file size small and retrieval fast. Each `zarr.Array` will have the same chunking as the corresponding `dask.Array`.

`y` is computed when we call `to_zarr`.

Excitingly, the `zarr.Array` is quite literally an array as it supports much of the NumPy functionality, just out-of-core. So in the previous code block, we could have written the random values immediately to the `*/X` components, and then transformed them into the `*/y` components. However, this would have been slower both due to the high cost of memory retrieval from disk and a lack of parallelization.

In [4]:
da.to_zarr(X[:N_TRN, :], 'data.zarr', component='train/X',)
da.to_zarr(X[N_TRN:, :], 'data.zarr', component='test/X')
da.to_zarr(y[:N_TRN], 'data.zarr', component='train/y')
da.to_zarr(y[N_TRN:], 'data.zarr', component='test/y')

## Data Reading and Model Fitting
We load the data from our Zarr file, and create an XGBoost regressor to predict future values. Given time, I hope to add examples for how to clean data with Dask.

### Data Reading with Dask

Dask effortlessly reads from Zarr files with the `from_zarr` function. As with writing, chunk size is preserved between arrays.

If you don't use a standard file format with a Dask function for reading either through Dask [arrays](https://docs.dask.org/en/latest/array-api.html#create-and-store-arrays) or [data frames](https://docs.dask.org/en/latest/dataframe-api.html#create-dataframes), I recommend using Dask's [delayed](https://docs.dask.org/en/latest/delayed.html) api.
Wrap the function to load your data with the `delayed` function, convert the data to an array format, then move it to a `dask.Array`. This will maximize the efficiency of your code relative to Dask.

In [5]:
X_train = da.from_zarr('data.zarr', component='train/X')
X_test = da.from_zarr('data.zarr', component='test/X')
y_train = da.from_zarr('data.zarr', component='train/y')
y_test = da.from_zarr('data.zarr', component='test/y')

### Model Fitting with XGBoost

For those unfamiliar, XGBoost is a machine learning package that provides access to several forms of gradient boosted models. Fortunately, the Python package exposes the Scikit-Learn api so you only need to learn the [math](https://xgboost.readthedocs.io/en/stable/tutorials/model.html) and the [parameters](https://xgboost.readthedocs.io/en/stable/parameter.html).

Unfortunately, there are some issues with Dask and XGBoost's interoperability. In particular, XGBoost's Dask models (e.g. `DaskXGBRegressor`) don't work with Dask's model selection objects (e.g. `RandomizedSearchCV`). Luckily, I found the [bug](https://github.com/dask/dask-ml/issues/849) causing this issue, and hopefully it will be fixed in a future update. Again, we are working with Dask v2022.4.0 and Dask_ML v2022.1.22.

As a result of this bug, there are currently two options.
1. Use `DaskXGB*` with Scikit-Learn's `RandomizedSearchCV` or `GridSearchCV`.
2. Use `XGB*` with Dask's `RandomizedSearchCV` or `GridSearchCV`.

The first option is preferable when you have big data and a small parameter search, the second is preferable when you have small data and a large parameter search. Now, the probabilists among us might point out that with large data, we could subset and search on a medium portion of the data, then train our final model on a larger portion. But, operations researchers will point out that regularization is dependent upon data size, so regularizer values selected on a medium subset should be suboptimal on the big train set. There are ways around this but these are beyond the scope of this notebook.

Note that we use the DART, or Drop-out Additive Regression Trees booster. At each training step, this booster sets the coefficient of any tree in the linear combination to 0 with probability `rate_drop`, more info can be found in Rashmi and Gilad-Bachrach ([2015](https://proceedings.mlr.press/v38/korlakaivinayak15.pdf)).

In general, when trying to constrain a hyperparameter search, as we are doing here for simplicity, my philosophy is to take each aspect of the model and select my favorite hyperparameter for regularizing it, then tune those hyper-parameters. So here we are using a DART booster and tweaking `rate_drop` to regularize the ensemble, with `max_depth` to regularize individual trees, and `lambda` (L2 shrinkage) for regularizing the complexity. Given enough time it would be wise to explore combinations of `eta` (learning rate) and `rate_drop`, `max_depth` and `max_leaves`, or `lambda` and `alpha` (L1 shrinkage).

I like to tune the chosen hyper-parameters because I believe that they lead to more compact trees, and smaller models. While Tan and Le's EfficientNet ([2019](https://arxiv.org/abs/1905.11946), [2021](https://arxiv.org/abs/2104.00298)) is a convolutional neural network, I encourage everyone to consider making their models as efficient as possible. In the case of a boosted tree model, that means minimizing the sum of the depths over all of the tree's. Unlike `eta` which decreases the impact of successive trees, `rate_drop` uses randomness as the regularization. In theory, this randomness produces more robust estimators, leading to fewer trees. Similarly, `max_depth` directly minimizes the tree depth, and `lambda` hinders model growth. To further optimize efficiency, we could select our final model based on the product of the final RMSE and the sum of tree depths. We will skip this for simplicity.

In [6]:
lvl = da.arange(1, 4).compute() # NumPy array
params = {
    'max_depth': lvl+2,
    'rate_drop': 0.015*lvl,
    'lambda': 0.5*lvl
}

## Buggy Option: distribute regression and search
# estim = xgb.dask.DaskXGBRegressor( booster='dart')
# model = dcv.RandomizedSearchCV(estim, params, n_iter=3)
# model.fit(X_train, y_train, early_stopping_rounds=10)


## Option 2: distribute search not regression
# estim = xgb.XGBRegressor(booster='dart')
# model = dcv.RandomizedSearchCV(estim, params, n_iter=3)
# model.fit(X_train, y_train, early_stopping_rounds=10)

## Option 1: distribute regression not search
estim = xgb.dask.DaskXGBRegressor(booster='dart', n_estimators=10) # set n_estimators and n_iter
model = sklcv.RandomizedSearchCV(estim, params, n_iter=1)          # to larger number for real training
model.fit(X_train, y_train) # SKLearn's RandomizedSearchCV doesn't work with early_stopping_rounds

/Users/chadgueli/miniconda3/envs/dayone/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/chadgueli/miniconda3/envs/dayone/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/chadgueli/miniconda3/envs/dayone/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/chadgueli/miniconda3/envs/dayone/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated an

RandomizedSearchCV(estimator=DaskXGBRegressor(booster='dart', n_estimators=10),
                   n_iter=1,
                   param_distributions={'lambda': array([0.5, 1. , 1.5]),
                                        'max_depth': array([3, 4, 5]),
                                        'rate_drop': array([0.015, 0.03 , 0.045])})

### Saving, Loading, and Testing Our Model

We begin by saving the model configuration with the `save_model` method. This stores all of the internal info for our model.

In [7]:
model.best_estimator_.save_model('smallmodel.json')

/Users/chadgueli/miniconda3/envs/dayone/lib/python3.9/site-packages/xgboost/sklearn.py:585: UserWarning: max_depth is not saved in Scikit-Learn meta.
  warnings.warn(str(k) + ' is not saved in Scikit-Learn meta.', UserWarning)


Now, to load the model, we need a compatible version of XGBoost, and a new model instance. We then call the `load_config` method on that instance to load our previous model. Finally, we can test our new model using predict.

In [8]:
new_model = xgb.XGBRegressor()
new_model.load_model('smallmodel.json')

rmse = lambda yt, yp: dmetrics.mean_squared_error(yt, yp, squared=False)
print(rmse(y_test, model.predict(X_test)))

1.7372491614873107
